## Connecting to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing PySPARK in Colab

[Pyspark version link](https://spark.apache.org/downloads.html)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()

In [ ]:
# # Second method to install PySPARK in  colab
# !pip install pyspark py4j

## Initialize  PySPARK

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

## Get spark UI

In [ ]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !sleep 5
# !curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

## Loading and checking Data

In [ ]:
Swiggy_DF = spark.read.csv("/content/drive/MyDrive/Swiggy/Swiggy_Analysis_Source_File.csv", header=True, inferSchema=True)

In [ ]:
Swiggy_DF.printSchema()

root
 |-- Restaurant Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: integer (nullable = true)
 |-- Has Table booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |--  Rating Stars out of 5: double (nullable = true)
 |-- Rating in text: string (nullable = true)
 |-- Price range: integer (nullable = true)
 |-- Votes: integer (nullable = true)



In [ ]:
Swiggy_DF.show(5)

+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|     Restaurant Name|     City|            Locality|     Cuisines|Average Cost for two|Has Table booking|Has Online delivery| Rating Stars out of 5|Rating in text|Price range|Votes|
+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|    Sultans of Spice|Bangalore|BluPetal Hotel, K...| North Indian|                1300|              Yes|                Yes|                   4.1|     Very Good|          3|  314|
|The Fatty Bao - A...|Bangalore|         Indiranagar|        Asian|                2400|              Yes|                Yes|                   4.7|     Excellent|          4|  591|
|                Toit|Bangalore|         Indiranagar|      Italian|                20

In [ ]:
#Replace the white Spaces with UnderScrol
columns=Swiggy_DF.columns
converter=lambda x:x.replace(' ','_')
columns=list(map(converter,columns))

In [ ]:
Swiggy_DF=Swiggy_DF.toDF(*columns)

In [ ]:
Swiggy_DF.show(5)

+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|     Restaurant_Name|     City|            Locality|     Cuisines|Average_Cost_for_two|Has_Table_booking|Has_Online_delivery|_Rating_Stars_out_of_5|Rating_in_text|Price_range|Votes|
+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+----------------------+--------------+-----------+-----+
|    Sultans of Spice|Bangalore|BluPetal Hotel, K...| North Indian|                1300|              Yes|                Yes|                   4.1|     Very Good|          3|  314|
|The Fatty Bao - A...|Bangalore|         Indiranagar|        Asian|                2400|              Yes|                Yes|                   4.7|     Excellent|          4|  591|
|                Toit|Bangalore|         Indiranagar|      Italian|                20

In [ ]:
Swiggy_DF.count()

258

## Populating Dimension Tables

#### 1) City_Dim

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,col

In [ ]:
City_df=Swiggy_DF.select('City').distinct()

In [ ]:
City_df.show()

+----------+
|      City|
+----------+
| Bangalore|
|     Patna|
|   Chennai|
|   Lucknow|
|    Mumbai|
|       Goa|
|   Kolkata|
|     Surat|
|      Pune|
|Chandigarh|
| Hyderabad|
|  Guwahati|
|    Jaipur|
+----------+



In [ ]:
wind=Window.orderBy(City_df['City'].asc())

In [ ]:
city_dm_Table=City_df.withColumn('City_Id',row_number().over(wind))

In [ ]:
city_dm_Table.printSchema()

root
 |-- City: string (nullable = true)
 |-- City_Id: integer (nullable = false)



In [ ]:
city_dm_Table.show()

+----------+-------+
|      City|City_Id|
+----------+-------+
| Bangalore|      1|
|Chandigarh|      2|
|   Chennai|      3|
|       Goa|      4|
|  Guwahati|      5|
| Hyderabad|      6|
|    Jaipur|      7|
|   Kolkata|      8|
|   Lucknow|      9|
|    Mumbai|     10|
|     Patna|     11|
|      Pune|     12|
|     Surat|     13|
+----------+-------+



In [ ]:
city_dm_Table.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\city_dm_Table_report")

#### 2) Restaurant Dimension Table

In [ ]:
rest_col=Swiggy_DF.select('Restaurant_Name').distinct()

In [ ]:
rest_col.show()

+--------------------+
|     Restaurant_Name|
+--------------------+
|            Chung Fa|
|             Bar Bar|
|Zolocrust - Hotel...|
|            Paradise|
|        Olive Bistro|
|            Flechazo|
|   The Night Factory|
|      Frozen Factory|
|    Sauticed Stories|
|        Uncle Jack's|
|               BarBQ|
|        The Food Lab|
|          Eat Street|
|           4 Seasons|
|     Taruveda Bistro|
|         Free Spirit|
|            Nawwarah|
|       Pind Balluchi|
|           Raj Rasoi|
| Three Dots & A Dash|
+--------------------+
only showing top 20 rows



In [ ]:
wind=Window.orderBy(rest_col['Restaurant_Name'].asc())

In [ ]:
restaurant_Dim_Table=rest_col.withColumn('Restaurant_Id',row_number().over(wind))

In [ ]:
restaurant_Dim_Table.show()

+--------------------+-------------+
|     Restaurant_Name|Restaurant_Id|
+--------------------+-------------+
|11th Avenue Cafe ...|            1|
|      145 Kala Ghoda|            2|
|          17 Degrees|            3|
|18 Degrees Resto ...|            4|
|      38 Degree East|            5|
|           4 Seasons|            6|
|  6 Ballygunge Place|            7|
|AB's - Absolute B...|            8|
|    Agent Jack's Bar|            9|
|            Angeethi|           10|
|             Antares|           11|
|              Apache|           12|
|Asia Kitchen by M...|           13|
|        Baba Au Rhum|           14|
|         Bansi Vihar|           15|
|             Bar Bar|           16|
|               BarBQ|           17|
|     Barbeque Nation|           18|
|      Barbeque Ville|           19|
|  Basil With A Twist|           20|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
restaurant_Dim_Table.count()

226

In [ ]:
restaurant_Dim_Table.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\ restaurant_dm_table_report")

#### 3) Locality Dimension Table

In [ ]:
local_col=Swiggy_DF.select('Locality').distinct()

In [ ]:
local_col.count()

137

In [ ]:
wins=Window.orderBy(local_col['Locality'].asc())

In [ ]:
locality_dm_table=local_col.withColumn('Locality_Id',row_number().over(wins))

In [ ]:
locality_dm_table.show()

+--------------------+-----------+
|            Locality|Locality_Id|
+--------------------+-----------+
|12th Square Build...|          1|
|Acropolis Mall, K...|          2|
|          Adajan Gam|          3|
|               Adyar|          4|
|            Aminabad|          5|
|           Anandpuri|          6|
|Anil Plaza, Chris...|          7|
|              Anjuna|          8|
|Anjuna Beach, Anjuna|          9|
|     Anna Nagar East|         10|
|             Arambol|         11|
|         Ashok Nagar|         12|
|               Athwa|         13|
|          Azad Nagar|         14|
|                Baga|         15|
|Balewadi High Str...|         16|
|          Ballygunge|         17|
|               Baner|         18|
|       Banjara Hills|         19|
|          Betalbatim|         20|
+--------------------+-----------+
only showing top 20 rows



In [ ]:
locality_dm_table.count()

137

In [ ]:
locality_dm_table.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\locality_dm_table_report")

#### 4) Cuisines dimension Table

In [ ]:
Swiggy_DF.printSchema()

root
 |-- Restaurant_Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average_Cost_for_two: integer (nullable = true)
 |-- Has_Table_booking: string (nullable = true)
 |-- Has_Online_delivery: string (nullable = true)
 |-- _Rating_Stars_out_of_5: double (nullable = true)
 |-- Rating_in_text: string (nullable = true)
 |-- Price_range: integer (nullable = true)
 |-- Votes: integer (nullable = true)



In [ ]:
Cousine_col=Swiggy_DF.select('Cuisines').distinct()

In [ ]:
Cousine_col.show() 

+--------------+
|      Cuisines|
+--------------+
|       Mexican|
|        Bakery|
|      European|
|Charcoal Grill|
|       Biryani|
|       Chinese|
|   Continental|
|       Mughlai|
|      Gujarati|
|    Hyderabadi|
|      Japanese|
|     Fast Food|
|          Goan|
|       Italian|
|      Lebanese|
| Modern Indian|
|       Bengali|
|  North Indian|
|      Desserts|
|  Healthy Food|
+--------------+
only showing top 20 rows



In [ ]:
wind=Window.orderBy(Cousine_col['Cuisines'].asc())
Cuisines_dm_table=Cousine_col.withColumn('Cuisines_id',row_number().over(wind))

In [ ]:
Cuisines_dm_table.show()

+--------------+-----------+
|      Cuisines|Cuisines_id|
+--------------+-----------+
|      American|          1|
|         Asian|          2|
|        Bakery|          3|
|       Bengali|          4|
|       Biryani|          5|
|          Cafe|          6|
|Charcoal Grill|          7|
|       Chinese|          8|
|   Continental|          9|
|      Desserts|         10|
|      European|         11|
|     Fast Food|         12|
|          Goan|         13|
|      Gujarati|         14|
|  Healthy Food|         15|
|    Hyderabadi|         16|
|       Italian|         17|
|      Japanese|         18|
|      Lebanese|         19|
| Mediterranean|         20|
+--------------+-----------+
only showing top 20 rows



In [ ]:
Cuisines_dm_table.count()

28

In [ ]:
Cuisines_dm_table.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\Cuisines_dm_table_report")

#### 5) Table Booking Dimension Table

In [ ]:
TableBooking_col=Swiggy_DF.select('Has_Table_booking').distinct()
TableBooking_col.show() 

+-----------------+
|Has_Table_booking|
+-----------------+
|               No|
|              Yes|
+-----------------+



In [ ]:
wind=Window.orderBy(TableBooking_col['Has_Table_booking'].desc())
TableBooking_dm_table=TableBooking_col.withColumn('Table_booking_Id',row_number().over(wind))

In [ ]:
TableBooking_dm_table.show()

+-----------------+----------------+
|Has_Table_booking|Table_booking_Id|
+-----------------+----------------+
|              Yes|               1|
|               No|               2|
+-----------------+----------------+



In [ ]:
TableBooking_dm_table.count()

2

In [ ]:
TableBooking_dm_table.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\ TableBooking_dm_table_report")

#### 6) Delivery_Available_Dm_Table

In [ ]:
Delivery_Available_col=Swiggy_DF.select('Has_Online_delivery').distinct()
Delivery_Available_col.show() 

+-------------------+
|Has_Online_delivery|
+-------------------+
|                 No|
|                Yes|
+-------------------+



In [ ]:
wind=Window.orderBy(Delivery_Available_col['Has_Online_delivery'].desc())
Delivery_Available_dm_table=Delivery_Available_col.withColumn('Delivery_ID',row_number().over(wind))

In [ ]:
Delivery_Available_dm_table.show()

+-------------------+-----------+
|Has_Online_delivery|Delivery_ID|
+-------------------+-----------+
|                Yes|          1|
|                 No|          2|
+-------------------+-----------+



In [ ]:
Delivery_Available_dm_table.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\Delivery_Available_dm_table_report")

#### 7) Rating Dimension Table

In [ ]:
Rating_col=Swiggy_DF.select('_Rating_Stars_out_of_5','Rating_in_text').distinct()
Rating_col.show() 

+----------------------+--------------+
|_Rating_Stars_out_of_5|Rating_in_text|
+----------------------+--------------+
|                   4.6|     Excellent|
|                   4.8|     Excellent|
|                   3.8|          Good|
|                   4.4|     Very Good|
|                   2.6|       Average|
|                   4.9|     Excellent|
|                   3.3|       Average|
|                   4.5|     Excellent|
|                   3.1|       Average|
|                   4.0|     Very Good|
|                   4.3|     Very Good|
|                   3.5|          Good|
|                   3.9|          Good|
|                   3.6|          Good|
|                   3.4|       Average|
|                   4.1|     Very Good|
|                   3.7|          Good|
|                   4.7|     Excellent|
|                   4.2|     Very Good|
+----------------------+--------------+



In [ ]:
wip=Window.orderBy(Rating_col['Rating_in_text'].asc())

In [ ]:
Rating_dm_table=Rating_col.withColumn('Rating_id',row_number().over(wip))

In [ ]:
Rating_dm_table.show()

+----------------------+--------------+---------+
|_Rating_Stars_out_of_5|Rating_in_text|Rating_id|
+----------------------+--------------+---------+
|                   2.6|       Average|        1|
|                   3.3|       Average|        2|
|                   3.1|       Average|        3|
|                   3.4|       Average|        4|
|                   4.6|     Excellent|        5|
|                   4.8|     Excellent|        6|
|                   4.9|     Excellent|        7|
|                   4.5|     Excellent|        8|
|                   4.7|     Excellent|        9|
|                   3.8|          Good|       10|
|                   3.5|          Good|       11|
|                   3.9|          Good|       12|
|                   3.6|          Good|       13|
|                   3.7|          Good|       14|
|                   4.4|     Very Good|       15|
|                   4.0|     Very Good|       16|
|                   4.3|     Very Good|       17|


In [ ]:
Rating_dm_table.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\Rating_dm_table_report")

## Fact table


In [ ]:
joined_Swiggy_DF=Swiggy_DF.join(city_dm_Table, ['City'])
joined_Swiggy_DF=joined_Swiggy_DF.join(restaurant_Dim_Table, ['Restaurant_Name'])
joined_Swiggy_DF=joined_Swiggy_DF.join(locality_dm_table, ['Locality'])
joined_Swiggy_DF=joined_Swiggy_DF.join(Cuisines_dm_table, ['Cuisines'])
joined_Swiggy_DF=joined_Swiggy_DF.join(TableBooking_dm_table, ['Has_Table_booking'])
joined_Swiggy_DF=joined_Swiggy_DF.join(Delivery_Available_dm_table, ['Has_Online_delivery'])
joined_Swiggy_DF=joined_Swiggy_DF.join(Rating_dm_table, ['_Rating_Stars_out_of_5'])
joined_Swiggy_DF.show()

+----------------------+-------------------+-----------------+-------------+--------------------+--------------------+---------+--------------------+--------------+-----------+-----+-------+-------------+-----------+-----------+----------------+-----------+--------------+---------+
|_Rating_Stars_out_of_5|Has_Online_delivery|Has_Table_booking|     Cuisines|            Locality|     Restaurant_Name|     City|Average_Cost_for_two|Rating_in_text|Price_range|Votes|City_Id|Restaurant_Id|Locality_Id|Cuisines_id|Table_booking_Id|Delivery_ID|Rating_in_text|Rating_id|
+----------------------+-------------------+-----------------+-------------+--------------------+--------------------+---------+--------------------+--------------+-----------+-----+-------+-------------+-----------+-----------+----------------+-----------+--------------+---------+
|                   4.1|                Yes|              Yes| North Indian|BluPetal Hotel, K...|    Sultans of Spice|Bangalore|                1300|  

In [ ]:
joined_Swiggy_DF.columns

['_Rating_Stars_out_of_5',
 'Has_Online_delivery',
 'Has_Table_booking',
 'Cuisines',
 'Locality',
 'Restaurant_Name',
 'City',
 'Average_Cost_for_two',
 'Rating_in_text',
 'Price_range',
 'Votes',
 'City_Id',
 'Restaurant_Id',
 'Locality_Id',
 'Cuisines_id',
 'Table_booking_Id',
 'Delivery_ID',
 'Rating_in_text',
 'Rating_id']

In [ ]:
columns_lst=['Price_range','Votes','Average_Cost_for_two','City_Id','Restaurant_Id','Locality_Id','Cuisines_id','Table_booking_Id','Delivery_ID','Rating_id']

In [ ]:
Fact_Swiggy_new=joined_Swiggy_DF.select(columns_lst)

In [ ]:
Fact_Swiggy_new.show()

In [ ]:
wip=Window.orderBy(Fact_Swiggy_new['Average_Cost_for_two'].asc())

In [ ]:
Fact_Swiggy=Fact_Swiggy_new.withColumn('Fact_id',row_number().over(wip))

In [ ]:
Fact_Swiggy.show()

In [ ]:
Fact_Swiggy.columns

['Price_range',
 'Votes',
 'Average_Cost_for_two',
 'City_Id',
 'Restaurant_Id',
 'Locality_Id',
 'Cuisines_id',
 'Table_booking_Id',
 'Delivery_ID',
 'Rating_id',
 'Fact_id']

In [ ]:
col_arrangement_lst=['Fact_id','City_Id','Locality_Id','Restaurant_Id','Cuisines_id',
                     'Average_Cost_for_two','Rating_id','Delivery_ID','Table_booking_Id',
                     'Votes','Price_range',]

In [ ]:
Fact_Swiggy=Fact_Swiggy.select(col_arrangement_lst)

In [ ]:
Fact_Swiggy.show()

+-------+-------+-----------+-------------+-----------+--------------------+---------+-----------+----------------+-----+-----------+
|Fact_id|City_Id|Locality_Id|Restaurant_Id|Cuisines_id|Average_Cost_for_two|Rating_id|Delivery_ID|Table_booking_Id|Votes|Price_range|
+-------+-------+-----------+-------------+-----------+--------------------+---------+-----------+----------------+-----+-----------+
|      1|      9|         48|           70|         10|                 200|        8|          2|               2|  360|          1|
|      2|     13|         13|          121|         27|                 250|       16|          2|               2|  123|          1|
|      3|     13|        132|          188|         10|                 250|       13|          2|               2| 1109|          1|
|      4|      5|        127|            6|          8|                 300|       13|          2|               2|  147|          3|
|      5|      9|          5|           79|         23|       

In [ ]:
Fact_Swiggy.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\Fact_Table_report")

## Count Of Restaurants ,City Wise

In [ ]:
joined_city_df=Fact_Swiggy.join(city_dm_Table, ['City_Id'])

In [ ]:
joined_city_df.show()

In [ ]:
restaurant_count=joined_city_df.groupBy("City").count()
restaurant_count.show()

+----------+-----+
|      City|count|
+----------+-----+
| Bangalore|   20|
|     Patna|   20|
|   Chennai|   20|
|   Lucknow|   21|
|    Mumbai|   20|
|       Goa|   20|
|   Kolkata|   20|
|     Surat|   20|
|      Pune|   20|
|Chandigarh|   18|
| Hyderabad|   18|
|  Guwahati|   21|
|    Jaipur|   20|
+----------+-----+



In [ ]:
restaurant_count.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\restaurant_count")

## City-wise Top 10 restaurant based on Avg cost for 2

In [ ]:
from pyspark.sql.functions import col

In [ ]:
joined_city_df=Fact_Swiggy.join(city_dm_Table, ['City_Id'])
joined_restaurant_df=joined_city_df.join(restaurant_Dim_Table, ['Restaurant_Id'])
joined_restaurant_df.show()

In [ ]:
window = Window.partitionBy(joined_restaurant_df['City']).orderBy(joined_restaurant_df['Average_Cost_for_two'].desc())

In [ ]:
City_wise_Top_10_restaurant = joined_restaurant_df.withColumn('row_no',row_number().over(window)).filter(col('row_no')<=10).select('City','Restaurant_Name','Average_Cost_for_two')

In [ ]:
City_wise_Top_10_restaurant.show(truncate=False)

+----------+--------------------------------+--------------------+
|City      |Restaurant_Name                 |Average_Cost_for_two|
+----------+--------------------------------+--------------------+
|Bangalore |The Fatty Bao - Asian Gastro Bar|2400                |
|Bangalore |Toit                            |2000                |
|Bangalore |Big Brewsky                     |1800                |
|Bangalore |Bombay Brasserie                |1500                |
|Bangalore |Koramangala Social              |1500                |
|Bangalore |Farzi Cafe                      |1500                |
|Bangalore |The Black Pearl                 |1400                |
|Bangalore |AB's - Absolute Barbecues       |1400                |
|Bangalore |Hoot                            |1400                |
|Bangalore |Sultans of Spice                |1300                |
|Chandigarh|Virgin Courtyard                |2200                |
|Chandigarh|TGI Friday's                    |1800             

In [ ]:
City_wise_Top_10_restaurant.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\City_wise_Top_10_restaurant")

## Top 10 restaurant based on Avg Votes

In [ ]:
joined_city_df=Fact_Swiggy.join(city_dm_Table, ['City_Id'])
joined_restaurant_df=joined_city_df.join(restaurant_Dim_Table, ['Restaurant_Id'])
joined_restaurant_df.show()

In [ ]:
window = Window.partitionBy(joined_restaurant_df['City']).orderBy(joined_restaurant_df['Votes'].desc())

In [ ]:
Top_10_restaurant_Avg  =joined_restaurant_df.withColumn("row",row_number().over(window)).filter(col("row") <= 10)

In [ ]:
Top_10_restaurant_Avg= Top_10_restaurant_Avg.select(['City','Restaurant_Name','Votes'])

In [ ]:
Top_10_restaurant_Avg.show(truncate=False)

+----------+--------------------------------+-----+
|City      |Restaurant_Name                 |Votes|
+----------+--------------------------------+-----+
|Bangalore |The Black Pearl                 |1070 |
|Bangalore |Onesta                          |677  |
|Bangalore |ECHOES Koramangala              |621  |
|Bangalore |The Fatty Bao - Asian Gastro Bar|591  |
|Bangalore |Truffles                        |532  |
|Bangalore |Onesta                          |520  |
|Bangalore |Eat Street                      |488  |
|Bangalore |AB's - Absolute Barbecues       |458  |
|Bangalore |Farzi Cafe                      |392  |
|Bangalore |Three Dots & A Dash             |365  |
|Chandigarh|Brooklyn Central                |618  |
|Chandigarh|The Night Factory               |535  |
|Chandigarh|Karim's                         |29   |
|Chandigarh|Virgin Courtyard                |17   |
|Chandigarh|OvenFresh                       |12   |
|Chandigarh|Mocha Bar                       |11   |
|Chandigarh|

In [ ]:
Top_10_restaurant_Avg.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\Top_10_restaurant")

## Top 10 restaurant based on Rating City wise

In [ ]:
joined_city_df=Fact_Swiggy.join(city_dm_Table, ['City_Id'])
joined_restaurant_df=joined_city_df.join(restaurant_Dim_Table, ['Restaurant_Id'])
joined_rating_df=joined_restaurant_df.join(Rating_dm_table, ['Rating_id'])
joined_rating_df.show()

In [ ]:
window = Window.partitionBy(joined_rating_df['City']).orderBy(joined_rating_df['_Rating_Stars_out_of_5'].desc())

In [ ]:
Top_10_restaurant_Rating =joined_rating_df.withColumn("row",row_number().over(window)).filter(col("row") <= 10).select(['City','Restaurant_Name','_Rating_Stars_out_of_5'])

In [ ]:
Top_10_restaurant_Rating.show(truncate=False)

+----------+--------------------------------+----------------------+
|City      |Restaurant_Name                 |_Rating_Stars_out_of_5|
+----------+--------------------------------+----------------------+
|Bangalore |Toit                            |4.8                   |
|Bangalore |The Fatty Bao - Asian Gastro Bar|4.7                   |
|Bangalore |ECHOES Koramangala              |4.7                   |
|Bangalore |Truffles                        |4.7                   |
|Bangalore |Onesta                          |4.6                   |
|Bangalore |AB's - Absolute Barbecues       |4.6                   |
|Bangalore |Onesta                          |4.6                   |
|Bangalore |Koramangala Social              |4.5                   |
|Bangalore |Big Brewsky                     |4.5                   |
|Bangalore |Flechazo                        |4.4                   |
|Chandigarh|Barbeque Nation                 |4.5                   |
|Chandigarh|Burgrill              

In [ ]:
Top_10_restaurant_Rating.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\Top_10_restaurant_Rating")

## Rating based on delivery availability City wise

In [ ]:
Delivery_Available_dm_table=Delivery_Available_col.withColumn('Delivery_ID',row_number().over(wind))

In [ ]:
joined_city_df=Fact_Swiggy.join(city_dm_Table, ['City_Id'])
joined_rating_df=joined_city_df.join(Rating_dm_table, ['Rating_id'])
joined_delivery_df=joined_rating_df.join(Delivery_Available_dm_table, ['Delivery_ID'])
joined_delivery_df.show()

In [ ]:
column_list = ["City","Has_Online_delivery"]
win_spec = Window.partitionBy(*column_list).orderBy(col("_Rating_Stars_out_of_5").desc())

In [ ]:
availability_City_wise=joined_delivery_df.withColumn("row",row_number().over(win_spec)).filter(col("row") <= 1).select(['City','Has_Online_delivery','_Rating_Stars_out_of_5'])

In [ ]:
availability_City_wise.show()

+----------+-------------------+----------------------+
|      City|Has_Online_delivery|_Rating_Stars_out_of_5|
+----------+-------------------+----------------------+
| Bangalore|                 No|                   4.8|
| Bangalore|                Yes|                   4.7|
|Chandigarh|                 No|                   4.5|
|Chandigarh|                Yes|                   4.5|
|   Chennai|                 No|                   4.9|
|   Chennai|                Yes|                   4.8|
|       Goa|                 No|                   4.8|
|  Guwahati|                 No|                   4.9|
| Hyderabad|                 No|                   4.9|
| Hyderabad|                Yes|                   4.7|
|    Jaipur|                 No|                   4.5|
|    Jaipur|                Yes|                   4.9|
|   Kolkata|                 No|                   4.9|
|   Kolkata|                Yes|                   4.6|
|   Lucknow|                 No|                

In [ ]:
availability_City_wise.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\availability_City_wise")

## 	Avg cost for 2 based on cuisine city wise

In [ ]:
Cuisines_dm_table=Cousine_col.withColumn('Cuisines_id',row_number().over(wind))

In [ ]:
joined_city_df=Fact_Swiggy.join(city_dm_Table, ['City_Id'])
joined_cuisines_df=joined_city_df.join(Cuisines_dm_table, ['Cuisines_id'])
joined_cuisines_df.show()

In [ ]:
window = Window.partitionBy(joined_cuisines_df['City']).orderBy(joined_cuisines_df['Average_Cost_for_two'].desc())

In [ ]:
cuisine_city_wise =joined_cuisines_df.withColumn("row",row_number().over(window)).select(['City','Cuisines','Average_Cost_for_two',])
cuisine_city_wise.show(40)

+----------+-------------+--------------------+
|      City|     Cuisines|Average_Cost_for_two|
+----------+-------------+--------------------+
| Bangalore| North Indian|                1300|
| Bangalore|        Asian|                2400|
| Bangalore|      Italian|                2000|
| Bangalore|     European|                1300|
| Bangalore|Modern Indian|                1500|
| Bangalore|       Bakery|                 800|
| Bangalore|    Fast Food|                 600|
| Bangalore|    Fast Food|                 600|
| Bangalore|  Continental|                 950|
| Bangalore|     American|                 800|
| Bangalore| North Indian|                1400|
| Bangalore| North Indian|                 400|
| Bangalore|  Continental|                1500|
| Bangalore|     European|                1400|
| Bangalore|        Asian|                1200|
| Bangalore|    Fast Food|                 600|
| Bangalore|  Continental|                1200|
| Bangalore|    Fast Food|              

In [ ]:
cuisine_city_wise.write.options(header='True', delimiter=',').csv(r"C:\Users\rohit\Desktop\swiggy\cuisine_city_wise")